In [1]:
import os

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
LANGCHAIN_TRACING_V2=os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"]=LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_API_KEY"]=LANGCHAIN_API_KEY
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

load model

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [7]:
#model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [8]:
message = [SystemMessage(content="Nice to meet you, Binh"), HumanMessage(content="Hi, how are you") ]

In [9]:
model.invoke(message)

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


AIMessage(content="I'm doing great, thank you for asking! I'm ready to assist you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--805a49c7-197e-4d96-a98e-43e2c61d432b-0', usage_metadata={'input_tokens': 13, 'output_tokens': 151, 'total_tokens': 164, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 125}})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTP

In [26]:
from langchain_core.output_parsers import StrOutputParser

In [28]:
parser = StrOutputParser()

In [27]:
respone = model.invoke(message)

In [29]:
parser.invoke(respone)

"I'm doing well, thank you for asking!\n\nHow about you?"

chaining with LCEL

In [30]:
chain = model | parser
chain.invoke(message)

"I'm doing great, thank you for asking!\n\nHow can I help you today?"

Prompting

In [31]:
from langchain_core.prompts import ChatPromptTemplate

In [32]:
system="translate the following into {language}"

In [35]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", "translate the following into {language}"),
     ("user", "{text}")]
)

In [36]:
prompt = prompt_template.invoke({"language":"english", "text":"Tôi đang học LangChain"})

In [37]:
parser.invoke(model.invoke(prompt))

'I am learning LangChain.'

Agents

In [19]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
#from langchain_tavily import TavilySearch
from langchain_core.messages import HumanMessage

In [11]:
memory = MemorySaver()

In [ ]:
search=TavilySearch( max_results=2)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_23424\1483482442.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search=TavilySearchResults( max_results=2)


In [13]:
tools=[search]

In [14]:
agent_executor=create_react_agent(model,tools,checkpointer=memory)

In [15]:
config = {"configurable": {"thread_id": "abc123"}}

In [16]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi im Binh! and i live in Hanoi")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Binh! It's nice to meet you. Hanoi is a beautiful city. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--80346058-5939-4906-b4e1-643ede6149fc-0', usage_metadata={'input_tokens': 84, 'output_tokens': 25, 'total_tokens': 109, 'input_token_details': {'cache_read': 0}})]}}
----


In [17]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats the weather where I live?")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Hanoi"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--27eaa326-b0ac-493c-870b-c4158117d0a5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Hanoi'}, 'id': '4d43201f-8de0-4534-9322-2150f93cacf9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 118, 'output_tokens': 79, 'total_tokens': 197, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 56}})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"title": "Past Weather in Hanoi, Vietnam — Yesterday and Last 2 Weeks", "url": "https://www.timeanddate.com/weather/vietnam/hanoi/historic", "content": "| 5:00 am | Image 26: Lots of rain. Mostly cloudy. | 75°F | Lots of rain. Mostly cloud